In [74]:
from pdf2image import convert_from_path
import os
import shutil
import json

In [75]:
functions = [
	{
		"q": {
			"text": "¿Qué parte de la siguiente ecuación debes resolver primero?",
			"math": "(3-1)+6*2+1"
		     },
		"correct": "(3-1)",
		"incorrect": ["6*2", "2+1"]
	},
	{
		"q": {
			"text": "¿Cuál es el siguiente paso?",
			"math": "2+6*2+1"
		     },
		"correct": "6*2",
		"incorrect": ["2+6", "2+1"]
	}
]

functions2 = [
	{
		"q": {
			"text": "¿Qué parte de la siguiente ecuación debes resolver primero?",
			"math": "8/2(2+2)"
		     },
		"correct": "(2+2)",
		"incorrect": ["8/2"]
	},
	{
		"q": {
			"text": "¿Cuál es el siguiente paso?",
			"math": "8/2(4)"
		     },
		"correct": "8/2",
		"incorrect": ["2(4)"]
	}
]

functions3 = [
	{
		"q": {
			"text": "¿Qué parte de la siguiente ecuación debes resolver primero?",
			"math": "3+5x4(4/4+1)"
		     },
		"correct": "4/4",
		"incorrect": ["3+5", "5*4", "4+1"]
	},
	{
		"q": {
			"text": "¿Cuál es el siguiente paso?",
			"math": "3+5*4(1+1)"
		     },
		"correct": "1+1",
		"incorrect": ["3+5", "5*4"]
	},
 	{
		"q": {
			"text": "¿Cuál es el siguiente paso?",
			"math": "3+5*4*2"
		     },
		"correct": "5*4*2",
		"incorrect": ["3+5"]
	}
]

functions4 = [
	{
		"q": {
			"text": "¿Qué parte de la siguiente ecuación debes resolver primero?",
			"math": "8+8(8*8)+5"
		     },
		"correct": "(8*8)",
		"incorrect": ["8+8", "8/8"]
	},
	{
		"q": {
			"text": "¿Cuál es el siguiente paso?",
			"math": "8+8*64+5"
		     },
		"correct": "8*64",
		"incorrect": ["8+8", "64+5"]
	}
]

sets = [functions, functions2, functions3, functions4]

In [76]:
def parse_latex(sets):
    for setIndex, set in enumerate(sets):
        levels = []
        for level, function in enumerate(set):
            level_dict = {
                "question": {
                    "type": "Image",
                    "value": f"{level}/question.png"
                },
                "answers": [{
                    "type": "Image",
                    "value": f"{level}/correct.png",
                    "answer": True
                }]
            }
            # Question
            qtex = f"""
                    \\documentclass{{slides}}
                    \\usepackage[paperheight=4.17in,paperwidth=6.25in]{{geometry}}
                    \\begin{{document}}
                    \\begin{{large}}
                    \\begin{{center}}
                        {function["q"]["text"]}
                        \\[ {function["q"]["math"]} \\]
                    \\end{{center}}
                    \\end{{large}}
                    \\end{{document}}
                    """
            with open("qtemp.tex", "w", encoding="utf8") as f:
                f.write(qtex)
                f.close()
            os.system("pdflatex qtemp.tex")
            qimage = convert_from_path("qtemp.pdf")

            try:
                os.mkdir(f"./game/assets/questions{setIndex}/{level}")
            except FileExistsError:
                pass
            try:
                qimage[0].save(f"./game/assets/questions{setIndex}/{level}/question.png", "PNG")
            except FileExistsError:
                pass
            
            # correct answer
            tex = f"""
                \\documentclass{{slides}}
                \\usepackage[paperheight=6.25in,paperwidth=6.25in]{{geometry}}
                \\begin{{document}}
                \\begin{{huge}}
                    \\[ {function['correct']} \\]
                \\end{{huge}}
                \\end{{document}}
                """
            with open("ctemp.tex", "w", encoding="utf8") as f:
                f.write(tex)
                f.close()
            os.system("pdflatex ctemp.tex")


            cimage = convert_from_path("ctemp.pdf")
            try:
                cimage[0].save(f"./game/assets/questions{setIndex}/{level}/correct.png", "PNG")
            except FileExistsError:
                pass
            
            # incorrect answers
            for i, answer in enumerate(function['incorrect']):
                incorrect_answer = {
                    "type": "Image",
                    "value": f"{level}/incorrect{i}.png",
                    "answer": False
                }
                level_dict["answers"].append(incorrect_answer)
                tex = f"""
                    \\documentclass{{slides}}
                    \\usepackage[paperheight=6.25in,paperwidth=6.25in]{{geometry}}
                    \\begin{{document}}
                    \\begin{{huge}}
                        \\[ {answer} \\]
                    \\end{{huge}}
                    \\end{{document}}
                    """
                with open("temp.tex", "w", encoding="utf8") as f:
                    f.write(tex)
                    f.close()
                os.system("pdflatex temp.tex")


                image = convert_from_path("temp.pdf")

                try:
                    image[0].save(f"./game/assets/questions{setIndex}/{level}/incorrect{i}.png", "PNG")
                except FileExistsError:
                    pass

            levels.append(level_dict)
        open(f"./game/assets/questions{setIndex}/questions.json", "w").write(json.dumps(levels))
        shutil.make_archive(f"./questions{setIndex}", 'zip', f"./game/assets/questions{setIndex}")


In [77]:
parse_latex(sets)